In [1]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams, get_associated_token_address
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import sys
sys.path.append('../../GlobalRepo/')
from global_repo import *


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [2]:
program_id = PublicKey("5hLihu5RjNSTPFFGP2nurV3xnJCeLj12WJXgphRDVMvE")
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

In [3]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [4]:
bump = 4;
avatar_mint = PublicKey("EVQ8rPoba7F8mEdB4TgGSXu4tnuMrfPkfzhrG3p7VjwL")
creator_pda, _ = PublicKey.find_program_address([b"avatar_mint_authority_pda"], AVATAR_PROGRAM_ID)
payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
collection_pda, _ = PublicKey.find_program_address([b"Dsol Dao_seed", bytes(creator_pda)], program_id)
payer_collection_pda, _ = PublicKey.find_program_address([b"payer_collection_dsol", bytes(payer_keypair.public_key), bytes(creator_pda)], program_id)
container_pda, container_pda_bump = PublicKey.find_program_address( [b"Dsol Dao_seed", bump.to_bytes(4, "big"), bytes(creator_pda), bytes(payer_keypair.public_key)], program_id);
rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
metadata_pda, _metadata_nonce = PublicKey.find_program_address([b"metadata", bytes(METAPLEX_PROGRAM_ID), bytes(avatar_mint)], METAPLEX_PROGRAM_ID);

accounts = [
    AccountMeta(payer_keypair.public_key, True, True),
    AccountMeta(creator_pda, False, False),
    AccountMeta(collection_pda, False, True),
    AccountMeta(container_pda, False, True),
    AccountMeta(avatar_mint, False, False),
    AccountMeta(instructions.get_associated_token_address(container_pda, avatar_mint), False, True),
    rent_account_meta, 
    spl_program_meta,
    sys_program_meta,
    AccountMeta(instructions.get_associated_token_address(payer_keypair.public_key, avatar_mint), False, True),
    AccountMeta(metadata_pda, False, False),
    AccountMeta(payer_collection_pda, False, True),



    sys_program_meta,
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    spl_program_meta,

]


In [5]:
transaction = Transaction()
transaction.add(
    TransactionInstruction(
        accounts,
        program_id,
        bytes([1,0,0,1,0,0,0,0,0,bump])
    )
)

f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/3BDLB4TbuBGizBHTeZmEiXSAmCkW7btqVxRwcCT1AoR5166S2Dmzt9QcCtLs4FVMgdDD6SVQGRX1udVQd8ry1nX8?cluster=devnet'